In [ ]:
import glob
import os
import tensorflow as tf
import numpy as np
from PIL import Image

In [ ]:
names = {
    "iguchi": 0,
    "ushio": 1,
    "kakizaki": 2,
    "kageyama": 3,
    "kato": 4,
    "saito": 5,
    "sasaki_k": 6,
    "sasaki_m": 7,
    "takase": 8 ,
    "takamoto":9 ,
    "higashimura": 10,
    "kanemura": 11,
    "kawata": 12,
    "kosaka": 13,
    "tomita": 14,
    "nibu": 15,
    "hamagishi": 16,
    "matsuda": 17,
    "miyata": 18,
    "watanabe": 19,
    "kamimura": 20
}

In [ ]:
learningPath = "images/learning/"
learningMemberPath = glob.glob(learningPath + "*")
for index, memberPath in enumerate(learningMemberPath):
    learningMemberPath[index] = memberPath.replace("\\", '/')
    
    # それぞれのメンバーの学習用画像を全部取得
    fileList = glob.glob(memberPath + "/" + "*")
    for imageFile in fileList:
        os.makedirs(memberPath + "/train/", exist_ok=True)
        shutil.move(imageFile, memberPath + "/train/" + imageFile.split("\\")[-1])

In [ ]:
# move test folder
for memberPath in learningMemberPath:
    fileList = glob.glob(memberPath + "/train/*")
    testRate = 0.2
    testNum = math.floor(len(fileList) * testRate)
    testFileList = random.sample(fileList, testNum)
    
    for imageFile in testFileList:
        os.makedirs(memberPath + "/test/", exist_ok=True)
        shutil.move(imageFile, memberPath + "/test/" + imageFile.split("\\")[-1])

In [ ]:
size = (128,128)

In [ ]:
trainDataList = []
testDataList = []
for memberFolder in learningMemberPath:
    trainFileList = glob.glob(memberFolder + "/train/*")
    testFileList = glob.glob(memberFolder + "/test/*")
    trainDataList.extend(trainFileList)
    testDataList.extend(testFileList)

In [ ]:
def CreateTensorflowReadFile(img_files, out_file):
    with tf.python_io.TFRecordWriter(out_file) as writer:
        for f in img_files:
            # ファイルを開く
            with Image.open(f) as image_object:
                image_object = image_object.resize(size, Image.LANCZOS)
                image = np.array(image_object)
                 
                height = image.shape[0]
                width = image.shape[1]
                image_raw = image.tostring()
                label = names[f.split('/')[-2]] # ラベルを取得
 
                example = tf.train.Example(features=tf.train.Features(feature={
                        "height": tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
                        "width": tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
                        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
                        "image": tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_object.tobytes()])),
                        "path": tf.train.Feature(bytes_list=tf.train.BytesList(value=[f.encode('utf-8')]))
                        }))
 
            # レコード書込
            writer.write(example.SerializeToString())

In [ ]:
CreateTensorflowReadFile(trainDataList , "train_tf_file_%sx%s.tfrecords" % size) # 書き込み

In [ ]:
CreateTensorflowReadFile(testDataList , "test_tf_file_%sx%s.tfrecords" % size) # 書き込み

In [ ]:
# 確認用
# output imagefile in tfrecord
count = 0
os.makedirs("check_tfRecords", exist_ok=True)
for record  in tf.python_io.tf_record_iterator("train_tf_file_%sx%s.tfrecords" % size):
    example = tf.train.Example()
    example.ParseFromString(record)  # バイナリデータからの読み込み
 
    height = example.features.feature["height"].int64_list.value[0]
    width = example.features.feature["width"].int64_list.value[0]
    label = example.features.feature["label"].int64_list.value[0]
    image = example.features.feature["image"].bytes_list.value[0]
    path = example.features.feature["path"].bytes_list.value[0]
 
    image = np.fromstring(image, dtype=np.uint8)
    image = image.reshape([height, width, 3])
    img = Image.fromarray(image, "RGB")
    img.save(os.path.join("check_tfRecords", "tfrecords_{0}-{1}.jpg".format(str(count), label)))
    count += 1